In [195]:
from keras.datasets import cifar10
import os
import numpy as np

from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import BatchNormalization
from keras.layers import Dropout
from keras.layers import Activation
import pickle
from keras import regularizers, optimizers
from keras.preprocessing.image import ImageDataGenerator
from keras.utils import np_utils

In [196]:
def unpickle(file):
    with open(file, 'rb') as f:
        data = pickle.load(f, encoding='latin-1')
    return data

In [197]:
def load_cifar10_data(data_dir):
    train_data = None
    train_labels = []
    for i in range(1, 6):
        data_dic = unpickle(data_dir + "/data_batch_{}".format(i))
        if i == 1:
            train_data = data_dic['data']
            train_labels += data_dic['labels']
        else:
            train_data = np.vstack((train_data, data_dic['data']))
            train_labels += data_dic['labels']
    test_data_dic = unpickle(data_dir + "/test_batch")
    test_data = test_data_dic['data']
    test_labels = test_data_dic['labels']

    train_data = train_data.reshape((len(train_data), 3, 32, 32))
    train_data = np.rollaxis(train_data, 1, 4)
    train_labels = np.array(train_labels)

    test_data = test_data.reshape((len(test_data), 3, 32, 32))
    test_data = np.rollaxis(test_data, 1, 4)
    test_labels = np.array(test_labels)

    return train_data, train_labels, test_data, test_labels



In [198]:
data_dir ='/Users/nagendrasingh/Desktop/cifar-10-batches-py'

x_train,y_train,x_test,y_test = load_cifar10_data(data_dir)



In [199]:
print(x_train.shape)


(50000, 32, 32, 3)


In [200]:
print(y_train.shape)



(50000,)


In [201]:
print(x_test.shape)


(10000, 32, 32, 3)


In [202]:
print(y_test.shape)

(10000,)


In [203]:
#normalizing the data
mean = np.mean(x_train,axis=(0,1,2,3))
std = np.std(x_train,axis=(0,1,2,3))
x_train = (x_train-mean)/(std+1e-7)
x_test = (x_test-mean)/(std+1e-7)

In [204]:
#one hot encoding of target labels
num_classes = 10
y_train = np_utils.to_categorical(y_train,num_classes)
y_test = np_utils.to_categorical(y_test,num_classes)

In [205]:
print(y_train.shape)

(50000, 10)


In [206]:
FilterNum = 32
weight_decay = 1e-4

In [207]:
model = Sequential()
model.add(Conv2D(FilterNum, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay), input_shape=x_train.shape[1:]))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(Conv2D(FilterNum, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))

model.add(Conv2D(2*FilterNum, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(Conv2D(2*FilterNum, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.3))

model.add(Conv2D(4*FilterNum, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(Conv2D(4*FilterNum, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Flatten())
model.add(Dense(num_classes, activation='softmax'))

model.summary()

W0629 18:05:05.298085 4735444416 deprecation_wrapper.py:119] From /anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0629 18:05:06.302186 4735444416 deprecation_wrapper.py:119] From /anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0629 18:05:06.614808 4735444416 deprecation_wrapper.py:119] From /anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0629 18:05:07.186919 4735444416 deprecation_wrapper.py:119] From /anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:174: The name tf.get_default_session is deprecated. Please use tf.compat.v1.get_default_session instead.

W0629 18:05:07.200326 4735444416 deprecation_wrapper

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 32, 32, 32)        896       
_________________________________________________________________
activation_1 (Activation)    (None, 32, 32, 32)        0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 32, 32, 32)        128       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 32, 32, 32)        9248      
_________________________________________________________________
activation_2 (Activation)    (None, 32, 32, 32)        0         
_________________________________________________________________
batch_normalization_2 (Batch (None, 32, 32, 32)        128       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 16, 16, 32)        0         
__________

In [208]:
model.compile(loss='categorical_crossentropy',
        optimizer="Adam",
        metrics=['accuracy'])

W0629 18:05:30.350902 4735444416 deprecation_wrapper.py:119] From /anaconda3/lib/python3.7/site-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.



In [209]:
batch_size = 64
data_augmentation = True

In [210]:
if data_augmentation :
    print("-------------Using Data augmentation------------")
     # This will do preprocessing and realtime data augmentation:
    datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=0,  # randomly rotate images in the range (degrees, 0 to 180)
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=True,  # randomly flip images
        vertical_flip=False)  # randomly flip images
    
    datagen.fit(x_train)
    model.fit_generator(datagen.flow(x_train, y_train, batch_size=batch_size),
                        steps_per_epoch=x_train.shape[0] // batch_size,
                        epochs=20,verbose=1,validation_data=(x_test,y_test))
    
else :
    print("-----Not Using Data augmentation---------------")
    model.fit(x_train, y_train,
              batch_size=batch_size*4,
              epochs=epochs,
              validation_data=(x_test, y_test),
              shuffle=True)
    
    
    

-------------Using Data augmentation------------


W0629 18:06:05.354043 4735444416 deprecation.py:323] From /anaconda3/lib/python3.7/site-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch 1/20
781/781 [==============================] - 536s 686ms/step - loss: 1.9197 - acc: 0.4156 - val_loss: 1.3476 - val_acc: 0.5418
Epoch 2/20
781/781 [==============================] - 582s 746ms/step - loss: 1.4828 - acc: 0.5472 - val_loss: 1.1651 - val_acc: 0.6204
Epoch 3/20
781/781 [==============================] - 445s 570ms/step - loss: 1.4700 - acc: 0.5782 - val_loss: 1.2138 - val_acc: 0.6439
Epoch 4/20
781/781 [==============================] - 460s 589ms/step - loss: 1.2378 - acc: 0.6351 - val_loss: 1.0580 - val_acc: 0.6710
Epoch 5/20
781/781 [==============================] - 528s 676ms/step - loss: 1.1650 - acc: 0.6576 - val_loss: 0.9953 - val_acc: 0.6963
Epoch 6/20
781/781 [==============================] - 1660s 2s/step - loss: 1.0181 - acc: 0.6937 - val_loss: 0.8874 - val_acc: 0.7312
Epoch 7/20
781/781 [==============================] - 529s 678ms/step - loss: 0.9203 - acc: 0.7207 - val_loss: 0.7976 - val_acc: 0.7608
Epoch 8/20
781/781 [==============================

In [212]:
model.save_weights("weight_file.h5")

In [211]:
score = model.evaluate(x_test,y_test,verbose=0)
print("Test score:",score[0])

print("Test accuracy:",score[1])

Test score: 0.6182007083415985
Test accuracy: 0.8414
